#THIS NOTEBOOK- CONTAINS BASIC READ OPERATIONS

#Create - Catalog(Workspace)/schema(database)/volume(Datalake (DBFS))/folder/mere data files

In [0]:
%sql
create catalog if not exists we47catalog;
create schema if not exists we47catalog.we47schema;
create volume if not exists we47catalog.we47schema.we47volume;

#Create Directory

In [0]:
dbutils.fs.mkdirs("/Volumes/we47catalog/we47schema/we47volume/we47dir1")

#Create some sample data

In [0]:
sample_data="welcome to databricks pyspark learning"
dbutils.fs.put("/Volumes/we47catalog/we47schema/we47volume/we47dir1/sample.txt",sample_data,True)
dbutils.fs.ls("/Volumes/we47catalog/we47schema/we47volume/we47dir1")    

In [0]:
dbutils.fs.head("/Volumes/we47catalog/we47schema/we47volume/we47dir1/sample.txt")


In [0]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()

**We are going to now learn - How to Read/Extract the data from the filesytem and load it into the distributed memory for further processing/load - using diffent methodologies/options from different sources(fs & db) and different builtin function formats (csv/json/orc/parquet/delta/tables)**

In [0]:
df1=spark.read.csv("/Volumes/we47catalog/we47schema/we47volume/we47dir1/sample.txt")
display(df1)
df1.show(20,True)

In [0]:
df1.printSchema()


##lets learn about CSV function

the below code by default considering c0,c1,c2.....cn as column name of the table as there is no column name given in this data

In [0]:
df1=spark.read.csv("/Volumes/we47catalog/we47schema/we47volume/we47dir1/custs")
display(df1)

the below code helps to add column name manually using .toDF method

In [0]:
df1=spark.read.csv("/Volumes/we47catalog/we47schema/we47volume/we47dir1/custs").toDF("custID","fname","lname","age","profession")
display(df1.limit(20))

the below code used another table(custs_header) which has its own column name, but the code takes the column name also as one of the data row of the table. to solev this, we have use header=True in the code

In [0]:
df1=spark.read.csv("/Volumes/we47catalog/we47schema/we47volume/we47dir1/custs_header")
display(df1.limit(20))

In [0]:
df1=spark.read.csv("/Volumes/we47catalog/we47schema/we47volume/we47dir1/custs_header",header=True)
display(df1.limit(20))

We use header option to consider the column name from the data itself and we can override the source given columns to our custom column names also by using header+ toDF()...

In [0]:
df1=spark.read.csv("/Volumes/we47catalog/we47schema/we47volume/we47dir1/custs_header",header=True).toDF("ID","F_name","L_name","Age","Occupation")
display(df1.limit(20))

In [0]:
df1.printSchema()

but if we see the datatype of all the column is string by default, but there are 2 integer type column in the table.

In [0]:
df1=spark.read.csv("/Volumes/we47catalog/we47schema/we47volume/we47dir1/custs_header",header=True,inferSchema=True).toDF("ID","F_name","L_name","Age","Occupation")
display(df1.limit(20))
df1.printSchema()

the below code used a table called custs_header_oth_del which has some other delimiter in the data (~) instead of the default one (,). hence, the code is considering all the data in single column

In [0]:
df1=spark.read.csv("/Volumes/we47catalog/we47schema/we47volume/we47dir1/custs_header_oth_del",header=True)
display(df1)

solving the above issue using the delimiter function in the code

In [0]:
df1=spark.read.csv("/Volumes/we47catalog/we47schema/we47volume/we47dir1/custs_header_oth_del",header=True,sep='~',inferSchema=True)
display(df1)
df1.printSchema()

##learning option, options, format and load method to read the file 

Using different options to create dataframe with csv and other module... (2 methodologies (spark.read.inbuiltfunction or spark.read.format(anyfunction).load("path")) with 3 ways of creating dataframes (pass parameters to the csv()/option/options))

In [0]:
#general method to read the data from a file
df1=spark.read.csv("/Volumes/we47catalog/we47schema/we47volume/we47dir1/custs_header_oth_del",header=True,sep='~',inferSchema=True)
print(df1.count())


In [0]:
#format and load method to read the data from a file
df1_bq=spark.read.format("csv").load("/Volumes/we47catalog/we47schema/we47volume/we47dir1/custs_header_oth_del",header=True,sep='~',inferSchema=True)
display(df1_bq)

In [0]:
#using format, load and option to read the data from a file
df1_bq=spark.read.format("csv").option("header","True").option("sep","~").load("/Volumes/we47catalog/we47schema/we47volume/we47dir1/custs_header_oth_del")
display(df1_bq)
df1_bq.printSchema()

In [0]:
#using format, load and options to read the data from a file
df1_bq=spark.read.options(header="True",sep="~",inferschema=True).format("csv").load("/Volumes/we47catalog/we47schema/we47volume/we47dir1/custs_header_oth_del")
display(df1_bq)
df1_bq.printSchema()

In [0]:
#another format without load and format function
df1_bq=spark.read.options(header="True",sep="~",inferschema=True).csv("/Volumes/we47catalog/we47schema/we47volume/we47dir1/custs_header_oth_del")
display(df1_bq)
df1_bq.printSchema()

##Reading data from multiple files & Multiple Path (We still have few more options)

In [0]:
df1_multiple_files=spark.read.csv("/Volumes/we47catalog/we47schema/we47volume/we47dir1/custs*",inferSchema=True,header=True).toDF("id","fname","lname","age","prof")
print(df1_multiple_files.count())

What if we have files in multiple names in multiple paths... Below code can read data from multiple paths and multiple files...

In [0]:
df1_multiple_files=spark.read.csv(["/Volumes/we47catalog/we47schema/we47volume/we47dir1/custs*","/Volumes/we47catalog/we47schema/we47volume/we47dir1/prospect"],inferSchema=True,header=True).toDF("id","fname","lname","age","prof")
print(df1_multiple_files.count())

getting data from different source systems of different regions (NY, TX, CA) into different landing pad (locations), how to access this data?

In [0]:
#recursiveFileLookup is a parameter that can be used to read data from multiple subdirectories
#pathGlobFilter is the parameter used for matching a given pattern under the main/subdirectories
df1_diff_subdirs=spark.read.csv("/Volumes/we47catalog/we47schema/we47volume/we47dir1",inferSchema=True,recursiveFileLookup=True,pathGlobFilter="custs*").toDF("id","fname","lname","age","prof")
print(df1_diff_subdirs.count())

##To provide schema (columnname & datatype), what are the 2 basic options available that we learned so far ? inferSchema/toDF<br>
We are going to learn additionally 2 more options to handle schema (colname & datatype)?

###1. Using simple string format of define schema.

In [0]:
datastruct="id int,fname string,lname string, age int,prof string"
df1=spark.read.schema(datastruct).csv("/Volumes/we47catalog/we47schema/we47volume/we47dir1/custs")
display(df1)
df1.printSchema()

###2. Using structure type to define schema(IMPORTANT)

In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType
custom_schema=StructType([StructField("id",IntegerType(),False),StructField("fname",StringType()),StructField("lname",StringType()),StructField("age",IntegerType()),StructField("profession",StringType())])
df1=spark.read.schema(custom_schema).csv("/Volumes/we47catalog/we47schema/we47volume/we47dir1/custs")
display(df1.limit(10))
df1.printSchema()

##Reading data from different builtin formats (csv, json, xml, orc, parquet, delta, table)

###1.Reading data from csv

In [0]:
df1=spark.read.csv("/Volumes/we47catalog/we47schema/we47volume/serialized_compressed_data_sources/csv_targetdata/",header=True,sep="~",inferSchema=True)
display(df1.limit(10))
df1.printSchema()

###2.Reading data from json

In [0]:
df1=spark.read.format("json").load("/Volumes/we47catalog/we47schema/we47volume/serialized_compressed_data_sources/json_targetdata/")
display(df1.limit(10))
df1.printSchema()

###3.Reading data from xml

In [0]:
df1=spark.read.xml("/Volumes/we47catalog/we47schema/we47volume/serialized_compressed_data_sources/xml_targetdata/",rowTag="customer")
display(df1.limit(10))
df1.printSchema()

###4.Reading data from Serialized format - orc

In [0]:
df1=spark.read.orc("/Volumes/we47catalog/we47schema/we47volume/serialized_compressed_data_sources/orc_targetdata/")
display(df1.limit(10))
df1.printSchema()

###5.Reading data from Serialized format - parquet

In [0]:
df1=spark.read.parquet("/Volumes/we47catalog/we47schema/we47volume/serialized_compressed_data_sources/parquet_targetdata/")
display(df1.limit(10))
df1.printSchema()

###6.Reading data from Serialized format - delta(parquet)

In [0]:
df1=spark.read.format("delta").load("/Volumes/we47catalog/we47schema/we47volume/serialized_compressed_data_sources/delta_targetdata/")
display(df1.limit(10))
df1.printSchema()

###7.Reading data from table - delta

In [0]:
df1=spark.read.table("default.customertbl")
display(df1.limit(10))
df1.printSchema()

In [0]:
df2=df1.select("age","custid","fname","lname","profession").filter("age=55")
display(df2.limit(10))

In [0]:
df2=df1.select("age","custid","fname","lname","profession").filter("profession= 'Pilot' ")
display(df2)